In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# set seed for reproducibility
random_seed = 1

In [2]:
df = pd.read_csv('fred_230718.csv', index_col='Date', parse_dates=True)
df = df.asfreq('QS')
earliest_date = '1963-01-01'
latest_date = '2021-10-01'
# # filter df index to be between earliest_date and latest_date
df = df.loc[(df.index >= earliest_date) & (df.index <= latest_date)]
df.dropna(axis=1, inplace=True)
df.head()

,GDP,GDPC1,GDPPOT,CPIAUCSL,CPILFESL,GDPDEF,M1V,M2V,DFF,UNRATE,...,MANEMP,DSPIC96,PCE,PCEDG,PSAVERT,DSPI,INDPRO,HOUST,GPDI,MSPUS
Date,,,,,,,,,,,,,,,,,,,,,
1963-01-01,621.672,3628.306,3662.738125,30.44,31.5,17.134,4.178,1.690,3.00,5.7,...,15545.0,2541.1,374.4,53.1,10.9,430.0,26.0448,1244.0,99.689,17800.0
1963-04-01,629.752,3669.020,3701.698767,30.48,31.7,17.164,4.194,1.675,3.00,5.7,...,15602.0,2547.1,376.4,53.2,10.7,431.1,26.7473,1689.0,101.650,18000.0
1963-07-01,644.444,3749.681,3741.388301,30.69,31.8,17.187,4.248,1.680,3.00,5.6,...,15646.0,2572.6,384.4,55.5,10.1,438.0,27.0445,1614.0,104.612,17900.0
1963-10-01,653.938,3774.264,3781.880559,30.75,32.0,17.326,4.269,1.672,3.50,5.5,...,15714.0,2617.3,386.0,54.2,11.5,447.0,27.5578,1779.0,107.189,18500.0
1964-01-01,669.822,3853.835,3822.450115,30.94,32.2,17.381,4.345,1.685,3.25,5.6,...,15715.0,2652.8,396.8,57.9,10.7,455.3,27.8820,1603.0,110.474,18500.0


set target and create, train, validate, and test datasets and then scale and transform them so they will work better with the neural network

In [3]:
target = 'MSPUS'
y = df[target]
X = df.drop(columns=[target]).shift(1).dropna()
y = y.loc[X.index] # Make sure y and X have the same rows after dropna

# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=random_seed)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)  # validation data should also be scaled
X_test_scaled = scaler.transform(X_test)

# Log-transform the target variable
y_train_log = np.log1p(y_train)
y_valid_log = np.log1p(y_valid)

let's create functions to train and evaluate models

In [4]:
def train_model(X_train, y_train, 
                X_valid, y_valid,
                layer_sizes=[100, 100], 
                activation="relu", 
                kernel_initializer="he_normal", 
                learning_rate=0.001, 
                epochs=100,
                batch_norm=False,
                l1_l2=False,
                l1=.01,
                l2=.01):

    # Create a sequential model
    model = tf.keras.Sequential()

    # Add batch normalization and dense layers according to the layer_sizes
    for size in layer_sizes:
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        if l1_l2:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer
                                            , kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        else:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer))

    # Add a final Dense layer with no activation
    model.add(tf.keras.layers.Dense(1))

    # Create the optimizer with the custom learning rate
    sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(loss="mse", optimizer=sgd)

    # Train the model using the scaled data
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid))
    
    return model

In [5]:
def evaluate_model(model, X_test, y_test, log_target=False):
    # When predicting, transform the predictions back
    y_pred = model.predict(X_test)
    if log_target:
        y_pred = np.expm1(y_pred).flatten()  # inverse of np.log1p(), make it 1D


    # compute the RMSE on the original scale
    mse = np.mean(tf.keras.losses.MSE(y_test, y_pred))
    print('Test set MSE:', mse)
    rmse = np.sqrt(mse)
    print('Test set RMSE:', rmse)
    rmspe = (np.sqrt(np.mean(np.square((y_test - y_pred) / y_test)))) * 100
    print('Test set RMSPE (%):', rmspe)

Now we'll test some different architectures

In [6]:
print('no regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

no regularization
Epoch 1/100
5/5 [==============================] - 1s 44ms/step - loss: 84.1387 - val_loss: 31.2523
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 22.6037 - val_loss: 14.2730
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 12.1391 - val_loss: 10.9895
Epoch 4/100
5/5 [==============================] - 0s 9ms/step - loss: 8.6289 - val_loss: 9.2844
Epoch 5/100
5/5 [==============================] - 0s 9ms/step - loss: 6.6809 - val_loss: 8.4116
Epoch 6/100
5/5 [==============================] - 0s 9ms/step - loss: 5.4317 - val_loss: 7.4204
Epoch 7/100
5/5 [==============================] - 0s 9ms/step - loss: 4.5749 - val_loss: 7.0579
Epoch 8/100
5/5 [==============================] - 0s 9ms/step - loss: 3.9673 - val_loss: 6.6243
Epoch 9/100
5/5 [==============================] - 0s 9ms/step - loss: 3.5756 - val_loss: 6.6934
Epoch 10/100
5/5 [==============================] - 0s 9ms/step - loss: 3.2403 - val_loss: 6.1534
Ep

In [7]:
print('batch norm regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

batch norm regularization
Epoch 1/100
5/5 [==============================] - 1s 39ms/step - loss: 91.8188 - val_loss: 57.3372
Epoch 2/100
5/5 [==============================] - 0s 11ms/step - loss: 38.0203 - val_loss: 34.5477
Epoch 3/100
5/5 [==============================] - 0s 12ms/step - loss: 14.1495 - val_loss: 25.6259
Epoch 4/100
5/5 [==============================] - 0s 11ms/step - loss: 6.3922 - val_loss: 22.6237
Epoch 5/100
5/5 [==============================] - 0s 12ms/step - loss: 2.9500 - val_loss: 21.3117
Epoch 6/100
5/5 [==============================] - 0s 12ms/step - loss: 3.1586 - val_loss: 20.4079
Epoch 7/100
5/5 [==============================] - 0s 12ms/step - loss: 3.1177 - val_loss: 20.1138
Epoch 8/100
5/5 [==============================] - 0s 12ms/step - loss: 2.0903 - val_loss: 19.7233
Epoch 9/100
5/5 [==============================] - 0s 11ms/step - loss: 1.2863 - val_loss: 19.1145
Epoch 10/100
5/5 [==============================] - 0s 12ms/step - loss: 1.7014 

In [8]:
print('L1/L2 regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

L1/L2 regularization
Epoch 1/100
5/5 [==============================] - 0s 37ms/step - loss: 90.8356 - val_loss: 42.2560
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 36.6848 - val_loss: 37.0332
Epoch 3/100
5/5 [==============================] - 0s 11ms/step - loss: 32.7529 - val_loss: 34.2125
Epoch 4/100
5/5 [==============================] - 0s 10ms/step - loss: 30.6885 - val_loss: 33.1004
Epoch 5/100
5/5 [==============================] - 0s 10ms/step - loss: 29.2681 - val_loss: 32.2815
Epoch 6/100
5/5 [==============================] - 0s 10ms/step - loss: 28.2062 - val_loss: 31.1652
Epoch 7/100
5/5 [==============================] - 0s 10ms/step - loss: 27.3883 - val_loss: 30.8277
Epoch 8/100
5/5 [==============================] - 0s 11ms/step - loss: 26.7194 - val_loss: 30.5851
Epoch 9/100
5/5 [==============================] - 0s 10ms/step - loss: 26.2135 - val_loss: 29.9900
Epoch 10/100
5/5 [==============================] - 0s 11ms/step - loss: 25.805

In [9]:
print('batch norm & L1/L2 regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

batch norm & L1/L2 regularization
Epoch 1/100
5/5 [==============================] - 1s 41ms/step - loss: 94.9053 - val_loss: 61.2724
Epoch 2/100
5/5 [==============================] - 0s 11ms/step - loss: 53.3000 - val_loss: 46.9972
Epoch 3/100
5/5 [==============================] - 0s 11ms/step - loss: 36.9144 - val_loss: 42.4953
Epoch 4/100
5/5 [==============================] - 0s 11ms/step - loss: 29.2727 - val_loss: 40.0248
Epoch 5/100
5/5 [==============================] - 0s 12ms/step - loss: 26.9551 - val_loss: 38.3869
Epoch 6/100
5/5 [==============================] - 0s 11ms/step - loss: 26.2159 - val_loss: 38.9920
Epoch 7/100
5/5 [==============================] - 0s 12ms/step - loss: 23.7855 - val_loss: 38.2659
Epoch 8/100
5/5 [==============================] - 0s 11ms/step - loss: 23.8341 - val_loss: 38.1982
Epoch 9/100
5/5 [==============================] - 0s 12ms/step - loss: 23.5903 - val_loss: 37.5987
Epoch 10/100
5/5 [==============================] - 0s 11ms/step -

In [10]:
model = train_model(epochs=500, layer_sizes=[100, 100, 100], X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

Epoch 1/500
5/5 [==============================] - 1s 49ms/step - loss: 120.6202 - val_loss: 88.1180
Epoch 2/500
5/5 [==============================] - 0s 12ms/step - loss: 70.2123 - val_loss: 70.0089
Epoch 3/500
5/5 [==============================] - 0s 13ms/step - loss: 50.3130 - val_loss: 65.0032
Epoch 4/500
5/5 [==============================] - 0s 13ms/step - loss: 42.3309 - val_loss: 62.9935
Epoch 5/500
5/5 [==============================] - 0s 13ms/step - loss: 39.7752 - val_loss: 62.9605
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 37.6366 - val_loss: 63.2563
Epoch 7/500
5/5 [==============================] - 0s 14ms/step - loss: 37.2086 - val_loss: 63.5868
Epoch 8/500
5/5 [==============================] - 0s 16ms/step - loss: 37.1155 - val_loss: 63.1984
Epoch 9/500
5/5 [==============================] - 0s 14ms/step - loss: 36.3437 - val_loss: 63.9916
Epoch 10/500
5/5 [==============================] - 0s 13ms/step - loss: 36.8389 - val_loss: 63.215

In [11]:
model = train_model(epochs=1_000, layer_sizes=[100, 100, 100, 100], X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

Epoch 1/1000
5/5 [==============================] - 1s 59ms/step - loss: 130.8261 - val_loss: 127.5651
Epoch 2/1000
5/5 [==============================] - 0s 14ms/step - loss: 78.6194 - val_loss: 103.2864
Epoch 3/1000
5/5 [==============================] - 0s 13ms/step - loss: 60.6630 - val_loss: 95.1316
Epoch 4/1000
5/5 [==============================] - 0s 13ms/step - loss: 53.3632 - val_loss: 92.4517
Epoch 5/1000
5/5 [==============================] - 0s 13ms/step - loss: 51.3319 - val_loss: 90.8049
Epoch 6/1000
5/5 [==============================] - 0s 13ms/step - loss: 50.6987 - val_loss: 88.9517
Epoch 7/1000
5/5 [==============================] - 0s 13ms/step - loss: 49.6738 - val_loss: 89.5176
Epoch 8/1000
5/5 [==============================] - 0s 12ms/step - loss: 49.9208 - val_loss: 88.3966
Epoch 9/1000
5/5 [==============================] - 0s 12ms/step - loss: 49.8291 - val_loss: 87.0021
Epoch 10/1000
5/5 [==============================] - 0s 13ms/step - loss: 49.6946 - val_

In [ ]:
import time

# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a more complex DNN model
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    # # this one was actually slower on GPU, batch size wasn't specified either
    # return tf.keras.models.Sequential([
    #     tf.keras.layers.Flatten(input_shape=(28, 28)),
    #     tf.keras.layers.Dense(128, activation='relu'),
    #     tf.keras.layers.Dropout(0.2),
    #     tf.keras.layers.Dense(10)
    # ])

# Train and evaluate the model, returning the time taken
def train_and_evaluate(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    start_time = time.time()
    # model.fit(x_train, y_train, epochs=5)
    model.fit(x_train, y_train, epochs=5, batch_size=512)
    end_time = time.time()

    model.evaluate(x_test, y_test, verbose=2)
    
    return end_time - start_time

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
    with tf.device('/GPU:0'):  # train with GPU
        model = create_model()
        gpu_time = train_and_evaluate(model, x_train, y_train, x_test, y_test)
        print("Time taken to train with GPU: {:.2f} seconds".format(gpu_time))
        
    with tf.device('/CPU:0'):  # train with CPU
        model = create_model()
        cpu_time = train_and_evaluate(model, x_train, y_train, x_test, y_test)
        print("Time taken to train with CPU: {:.2f} seconds".format(cpu_time))
        
    speed_increase = (cpu_time - gpu_time) / cpu_time * 100
    print("GPU is {:.2f}% faster than CPU".format(speed_increase))
else:
    print("GPU not available, training with CPU only")
    model = create_model()
    cpu_time = train_and_evaluate(model, x_train, y_train, x_test, y_test)
    print("Time taken to train with CPU: {:.2f} seconds".format(cpu_time))